# Context Manager

We use it when read a file that we want to close, a connection to a database that we need to close after a error occur, etc.

In [11]:
import logging
import sqlite3


def make_query(query: str):
    logging.basicConfig(level=logging.INFO)
    connection = sqlite3.connect("application.db")
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        logging.info(cursor.fetchall())
    finally:
        connection.close()

In [12]:
make_query("SELECT * FROM blogs")

OperationalError: no such table: blogs

Instead of using try finally, we can use a context that do the same job:

In [13]:
import logging
import sqlite3


def make_query(query: str):
    logging.basicConfig(level=logging.INFO)
    with sqlite3.connect("application.db") as connection:
        cursor = connection.cursor()
        cursor.execute(query)
        logging.info(cursor.fetchall())

In [14]:
make_query("SELECT * FROM blogs")

OperationalError: no such table: blogs

# Creating our own context manager

Define a class with dunder methods ```__enter__``` and ```__exit__```:

In [15]:
import logging
import sqlite3


class SQLite:
    def __init__(self, file_name: str):
        self.file_name = file_name
        self.connection = sqlite3.connect(file_name)

    def __enter__(self):
        logging.info("Calling __enter__")
        return self.connection.cursor()

    def __exit__(self, error: Exception, value: object, traceback: object):
        logging.info("Calling __exit__")
        self.connection.commit()
        self.connection.close()

def make_query(query: str):
    logging.basicConfig(level=logging.INFO)
    with SQLite("application.db") as cursor:
        cursor.execute(query)
        logging.info(cursor.fetchall())

In [16]:
make_query("SELECT * FROM blogs")

INFO:root:Calling __enter__
INFO:root:Calling __exit__


OperationalError: no such table: blogs